# New TDM client demo

### Download and filter metadata with Pandas

In [ ]:
# Parameters:
dataset_id = "674bc524-aeae-f683-7b83-433dec676b5b"

In [ ]:
# Importing your dataset with a dataset ID
from tdm_package import tdm_client

dataset_metadata = tdm_client.get_metadata(dataset_id)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(dataset_metadata)

In [ ]:
print('Total documents', len(df))

In [ ]:
dataset_document_count = len(df)
print("Total documents", dataset_document_count)

In [ ]:
# Set the pandas option to show all columns
pd.set_option("max_columns", None) 

df.head() # Show the first five rows of our DataFrame

In [ ]:
id_list = df['id'].tolist()

In [ ]:
'http://www.jstor.org/stable/2871420' in id_list

In [ ]:
id_list

In [ ]:
# Drop each of these named columns
df = df.drop(['outputFormat', 'pageEnd', 'pageStart', 'datePublished', 'language'], axis=1)

In [ ]:
# Drop articles without an author
df = df.dropna(subset=['creator'])

In [ ]:
print("Original total", dataset_document_count)
print("Filtered total", len(df))

In [ ]:
# Examples for filtering the data based on the values found under 'title'

df = df[df.title != 'Review Article'] # Remove articles with title "Review Article"
df = df[df.title != 'Front Matter'] # Remove articles with title "Front Matter"
df = df[df.title != 'Back Matter'] # Remove articles with title "Back Matter"

# Remove articles with fewer than 3000 words, adjust or remove

df = df[df.wordCount > 3000] 

In [ ]:
print("Original total", dataset_document_count)
print("Filtered total", len(df))

In [ ]:
filtered_id_list = df["id"].tolist()

In [ ]:
df.groupby(['publicationYear'])['id'].agg('count').plot.bar(title='Documents by year', figsize=(20, 5), fontsize=12); 

In [ ]:
df.groupby(['publicationYear'])['pageCount'].agg('sum').plot.bar(title='Pages by decade', figsize=(20, 5), fontsize=12);

### Count word frequencies

In [ ]:
dataset_json_file = tdm_client.get_dataset(dataset_id)

In [ ]:
import json
import gzip
from collections import Counter

word_frequency = Counter()

In [ ]:
with gzip.open(dataset_json_file, "rb") as input_file:
    for row in input_file:
        document = json.loads(row)
        _id = document["id"]
        if _id in filtered_id_list:
            unigrams = document.get("unigramCount", [])
            for gram, count in unigrams.items():
                word_frequency[gram] += count

In [ ]:
word_frequency

In [ ]:
for gram, count in word_frequency.most_common(25):
    print(gram.ljust(20), count)

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words[:10]

In [ ]:
transformed_word_frequency = Counter()

In [ ]:
for document in tdm_client.dataset_reader(dataset_json_file):
    _id = document["id"]
    if _id in filtered_id_list:
        unigrams = document.get("unigramCount", [])
        for gram, count in unigrams.items():
            clean_gram = gram.lower()
            if clean_gram in stop_words:
                continue
            transformed_word_frequency[clean_gram] += count
        break

In [ ]:
transformed_word_frequency

In [ ]:
for gram, count in transformed_word_frequency.most_common(25):
    print(gram.ljust(20), count)

In [ ]:
df = pd.DataFrame(list(transformed_word_frequency.items())[:25], columns=["ngram", "count"])

In [ ]:
df.head()

In [ ]:
df.sort_values('count', ascending=True).plot.barh(title='Frequent words', figsize=(20, 10), fontsize=12, x="ngram", y="count");

### Significant terms

Run TFIDF on the first 10 documents in the filtered corpus.

In [ ]:
def process_token(token):
    token = token.lower()
    if token in stop_words:
        return
    if len(token) < 4:
        return
    if not(token.isalpha()):
        return
    return token

In [ ]:
import gensim

In [ ]:
# Limit to n documents. Set to None to do all

limit = 500

n = 0
documents = []
for document in tdm_client.dataset_reader(dataset_json_file):
    processed_document = []
    _id = document["id"]
    if _id in filtered_id_list:
        unigrams = document.get("unigramCount", [])
        for gram, count in unigrams.items():
            clean_gram = process_token(gram)
            if clean_gram is None:
                continue
            processed_document.append(clean_gram)
        if len(processed_document) > 0:
            documents.append(processed_document)
        n += 1
        if (limit is not None) and (n >= limit):
           break

In [ ]:
dictionary = gensim.corpora.Dictionary(documents)

In [ ]:
dict(list(dictionary.token2id.items())[0:10]) # Print the first ten tokens and their associated IDs.


In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in documents]

In [ ]:
model = gensim.models.TfidfModel(bow_corpus)

In [ ]:
corpus_tfidf = model[bow_corpus]

In [ ]:
rows = []
for doc in corpus_tfidf:
    for term_id, score in doc:
        rows.append([dictionary.get(term_id), score])

In [ ]:
df = pd.DataFrame(rows, columns=["ngram", "score"])

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
top_50 = df.sort_values("score", ascending=False).head(n=50)

In [ ]:
top_50.sort_values("score", ascending=True).plot.barh(title='Significant terms', figsize=(20, 10), fontsize=12, x="ngram", y="score");

 ### LDA topic modeling

In [ ]:
doc_count = len(id_list)
num_topics = 7 # Change the number of topics

# Remove terms that appear in less than 10% of documents and more than 75% of documents.
dictionary.filter_extremes(no_below=10 * .10, no_above=0.75)


In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in documents]

In [ ]:
# Train the LDA model.
model = gensim.models.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=num_topics
)

In [ ]:
for topic_num in range(0, num_topics):
    word_ids = model.get_topic_terms(topic_num)
    words = []
    for wid, weight in word_ids:
        word = dictionary.id2token[wid]
        words.append(word)
    print("Topic {}".format(str(topic_num).ljust(5)), " ".join(words))